**Video 資料位置**

- https://drive.google.com/file/d/1-9ngKHZ5QN2bb6kTRgKGNqjAguwSbtix/view?usp=sharing


**CNN Weights**

- https://drive.google.com/file/d/17Uem7LOIMwNvLSbkQSOEYc5cnghcbnl8/view?usp=sharing

In [1]:
#@title Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title 從Google Drive 匯入影片資料
train_data_path = '/content/drive/MyDrive/aidataset/df_train.zip' #@param {type:"string"}
cmd  = f'cp {train_data_path} ./df_train.zip'
! $cmd
! unzip df_train.zip

In [2]:
#@title 從Google Drive 匯入CNN Weights資料
train_data_path = '/content/drive/MyDrive/aidataset/cnn_weights.hdf5' #@param {type:"string"}
cmd  = f'cp {train_data_path} ./cnn_weights.hdf5'
! $cmd


In [5]:
#@title 安裝 facenet_pytorch
! pip install facenet_pytorch

     |████████████████████████████████| 1.9 MB 12.6 MB/s 


In [9]:
#@title 載入 CNN 模型
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.callbacks import CSVLogger
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.applications.nasnet import NASNetLarge
from matplotlib import pyplot as plt
from tensorflow.keras import backend as K
import numpy as np
import time
import argparse
from os.path import exists
from os import makedirs
import tensorflow.compat.v1.keras.backend as K
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

def cnn_model(model_name, img_size, weights):
    """
    Model definition using Xception net architecture
    """
    input_size = (img_size, img_size, 3)
    if model_name == "xception":
        baseModel = Xception(
            weights="imagenet",
            include_top=False,
            input_shape=(img_size, img_size, 3)
        )
    elif model_name == "iv3":
        baseModel = InceptionV3(
            weights="imagenet",
            include_top=False,
            input_shape=(img_size, img_size, 3)
        )
    elif model_name == "irv2":
        baseModel = InceptionResNetV2(
            weights="imagenet",
            include_top=False,
            input_shape=(img_size, img_size, 3)
        )
    elif model_name == "resnet":
        baseModel = ResNet50(
            weights="imagenet",
            include_top=False,
            input_shape=(img_size, img_size, 3)
        )


    headModel = baseModel.output
    headModel = GlobalAveragePooling2D()(headModel)
    headModel = Dense(512, activation="relu", kernel_initializer="he_uniform", name="fc1")(
        headModel
    )
    headModel = Dropout(0.4)(headModel)

    predictions = Dense(
        2,
        activation="softmax",
        kernel_initializer="he_uniform")(
        headModel
    )
    model = Model(inputs=baseModel.input, outputs=predictions)

    model.load_weights(weights + ".hdf5")
    print("Weights loaded...")
    model_lstm = Model(
        inputs=baseModel.input,
        outputs=model.get_layer("fc1").output
    )

    for layer in baseModel.layers:
        layer.trainable = True

    optimizer = Nadam(
        lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004
    )
    model.compile(
        loss="categorical_crossentropy",
        optimizer=optimizer,
        metrics=["accuracy"]
    )
    return model_lstm


In [10]:
#@title 利用CNN 擷取影片特徵
from facenet_pytorch import MTCNN
from os import listdir
from os.path import join
from glob import glob
#listdir()
# Create face detector
mtcnn = MTCNN(
    margin=40,
    select_largest=False,
    post_process=False,
    device="cuda:0"
)

train_dir = "./train/"
sub_directories = listdir(train_dir)

videos = []

for i in sub_directories:
    videos += glob(join(train_dir, i, "*.mp4"))
model_choice = 'xception' #@param ["xception", "iv3", "irv2", "resnet"]
# Loading model for feature extraction
model = cnn_model(
    model_name=model_choice,
    img_size=160, 
    weights = 'cnn_weights'
)

Instructions for updating:
Colocations handled automatically by placer.
83697664/83683744 [==============================] - 1s 0us/step
Weights loaded...


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/nadam.py:73: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Nadam, self).__init__(name, **kwargs)


In [14]:
#@title 準備好 Feautre 並存成npy 檔案

import cv2
from PIL import Image
features = []
counter = 0
labels = []

for video in videos:
    cap = cv2.VideoCapture(video)
    labels += [int(video.split("/")[-2])]

    batches = []

    while cap.isOpened() and len(batches) < 25:
        ret, frame = cap.read()
        if not ret:
            break

        h, w, _ = frame.shape
        if h >= 1080 and w >= 1920:
            frame = cv2.resize(
                frame,
                (640, 480),
                interpolation=cv2.INTER_AREA
            )

        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = Image.fromarray(frame)
        face = mtcnn(frame)

        try:
            face = face.permute(1, 2, 0).int().numpy()
            batches.append(face)
        except AttributeError:
            print("Image Skipping")

    cap.release()
    batches = np.array(batches).astype("float32")
    batches /= 255

    # fc layer feature generation
    predictions = model.predict(batches)

    features += [predictions]

    if counter % 50 == 0:
        print("Number of videos done:", counter)
    counter += 1

features = np.array(features)
labels = np.array(labels)

print(features.shape, labels.shape)

np.save("lstm_40fpv_data.npy", features)
np.save("lstm_40fpv_labels.npy", labels)

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Number of videos done: 0
Number of videos done: 50
Number of videos done: 100
Number of videos done: 150
(165, 25, 512) (165,)


In [15]:
#@title 將打包檔案搬回Google Drive
dst_path = '/content/drive/MyDrive/aidataset/' #@param {type:"string"}
cmd  = f'cp lstm_*.npy {dst_path}'
! $cmd